In [1]:
%load_ext autoreload
%autoreload 2
from tf_activation import DeepGraph
from tf_activation.models import mnist_cpcpff as mnist_model

import math
import random

import networkx as nx
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net
net, keep_prob = mnist_model.deepnn(x)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=net['y_conv']))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(net['y_conv'], 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [4]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, '../logdir/models/mnist_cpcpff_2000.0.ckpt')
    print(accuracy.eval(feed_dict={x: mnist.test.images[:1], y_:mnist.test.labels[:1], keep_prob: 1.0}))
    
    batch = mnist.train.next_batch(1)
    
    conv1 = {}
    conv1['W'] = sess.run(net['W_conv1'])
    conv1['i'] = sess.run(net['input'], feed_dict={x: mnist.test.images[:1]})
    conv1['o'] = sess.run(net['h_conv1'], feed_dict={x: mnist.test.images[:1]})
        
    pool1 = {}
    pool1['W'] = conv1['W']
    pool1['i'] = conv1['o']
    pool1['o'] = sess.run(net['h_pool1'], feed_dict={x: mnist.test.images[:1]})
        
    conv2 = {}
    conv2['W'] = sess.run(net['W_conv2'])
    conv2['i'] = pool1['o']
    conv2['o'] = sess.run(net['h_conv2'], feed_dict={x: mnist.test.images[:1]})
        
    pool2 = {}
    pool2['W'] = conv2['W']
    pool2['i'] = conv2['o']
    pool2['o'] = sess.run(net['h_pool2'], feed_dict={x: mnist.test.images[:1]})
        
    fc1 = {}
    fc1['W'] = sess.run(net['W_fc1'])
    fc1['i'] = pool2['o']
    fc1['o'] = sess.run(net['h_fc1'], feed_dict={x: mnist.test.images[:1]})
        
    fc2 = {}
    fc2['W'] = sess.run(net['W_fc2'])
    fc2['i'] = sess.run(net['h_fc1_drop'], feed_dict={x: mnist.test.images[:1], keep_prob: 1.0})
    fc2['o'] = sess.run(net['y_conv'], feed_dict={x: mnist.test.images[:1],  keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ../logdir/models/mnist_cpcpff_2000.0.ckpt
97.3755


In [5]:
seed = 1
random.seed(a=seed)
np.random.seed(seed=seed)
def build_elites(im, size=200, num_changes=100):
    elites = np.empty(shape=(size+1,im.shape[0],im.shape[1]))
    for i in range(size):
        idxs = np.random.choice(im.shape[1], size=num_changes)
        elite = np.copy(im[0,:])
        for idx in idxs:
            elite[idx] = elite[idx] + random.uniform(-elite[idx], 1-elite[idx])
        elites[i,:,:] = elite
    elites[size,:,:] = im
    return elites

In [ ]:
def train_elites(im, lab, trials=10000, size=200, num_changes=10, print_every=100):
    
    ret_elite = None
    with tf.Session() as sess:
        saver.restore(sess, '../logdir/models/mnist_cpcpff_2000.0.ckpt')
        im = np.reshape(im, (1,im.shape[0]))
        lab = np.reshape(lab, (1, lab.shape[0]))
        print('Initial error: {}'.format(cross_entropy.eval(feed_dict={x: im, y_:lab, keep_prob: 1.0})))

        trial = 0
        trial_im = im
        while trial < trials:
            elite_set = build_elites(trial_im, size=size, num_changes=num_changes)
            results = np.zeros(shape=(elite_set.shape[0]))
            for i in range(elite_set.shape[0]):
                results[i] = cross_entropy.eval(feed_dict={x: elite_set[i,:, :], y_:lab, keep_prob: 1.0})
            min_idx = np.argmin(results)
            if trial % print_every == 0:
                print('Min cross-entropy in trial {}: {}'.format(trial, results[min_idx]))
            trial_im = elite_set[min_idx,:,:]
            trial = trial + 1
        ret_elite = trial_im
            
    return ret_elite
            

In [ ]:
im = mnist.test.images[0]
lab = mnist.test.labels[0]
elite = train_elites(im, lab, num_changes=100)

INFO:tensorflow:Restoring parameters from ../logdir/models/mnist_cpcpff_2000.0.ckpt


In [ ]:
plt.imshow(np.reshape(elite,[28,28]), interpolation="nearest", cmap="gray")